# Estructura

In [62]:
from os import listdir
from os.path import join, dirname, basename, isdir
import pandas as pd

In [75]:
path = '../../scikit-learn-master/sklearn'

In [76]:
from collections import deque

In [124]:
q = deque()
q.append(path)

d = dict()
while q:
    current = q.pop()
    name = basename(current)
    selfd = {
        'isdir': isdir(current), 
        'parent': dirname(current), 
        'name': name,
        'ispy': current.endswith('.py')
    }
    if isdir(current):
        children = [join(current, f) for f in listdir(current)]
        q.extend(children)
        selfd['children'] = children
    else:
        selfd['children'] = None
    d[current] = selfd

In [125]:
df = pd.DataFrame.from_dict(d, orient='index')
df.index.name = 'fullpath'

In [126]:
df.to_csv('sklearn_structure.csv')

In [127]:
df.to_pickle('sklearn_structure.pkl')

In [135]:
def make_tree(fullpath):
    info = df.loc[fullpath]
    tree = info[['name', 'ispy']].to_dict()
    tree['name'] = info['name']
    if info['children']:
        tree['children'] = [make_tree(c) for c in info['children']]
    return tree

# Importancia de nodos

In [137]:
files = df[df.ispy].copy()

In [138]:
def is_import_line(line):
    line = line.strip()
    return line.startswith('import ') or line.startswith('from ')

In [150]:
def get_imports(fullpath):
    return list(filter(is_import_line, open(fullpath).read().splitlines()))

In [151]:
imports = files[[]].reset_index()['fullpath'].apply(get_imports)